In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed May 20 08:59:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.utils.data as loader
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary

In [0]:
from torch.utils.data import Dataset
import gzip
import os
class ModelNet10GAN(Dataset):
    """
    Custom dataset for training a 3D-GAN, without using a variational autoencoder.
    I have not implemented download of dataset. Also keep a minimum RAM of 12 GB.
    """
    def __init__(self, dir="./", download=False):
        """
        Args:
            dir (string): Path in which you want the dataset 
                to be saved (keep a minimum space of 9 GBs).
            download (boolean): Set to True if you want to download. 
                Default=False.
        """
        self.dir=dir
        self.download=download
        if not self._check_exists():
            raise RuntimeError('chair.npy.gz not present in '+self.dir)
        with gzip.open(self.dir+'chair.npy.gz','rb') as f:
            self.arr=np.load(f)
        
        
 
    def __getitem__(self, ind):
        """
        Args:
            ind (int): Index of the sample you want. 
 
        Returns:
            Tensor: (torch.Tensor, Size: (1,64,64,64))
        """
        return torch.tensor(self.arr[ind+1])
 
    def __len__(self):
        return (self.arr.shape[0]-1)
 
    def _check_exists(self):
        return (os.path.exists(self.dir+"chair.npy.gz"))

In [0]:
# Storing device
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# Defining ModelNet10 dataset for GAN
dataset=ModelNet10GAN(dir='./')
# Defining batch-size for every iteration.
batchsize=64
# Loading dataset into dataloader
data_loader=loader.DataLoader(dataset, batch_size=batchsize)

In [0]:

# This is the input vector size. Default = 200 (as per paper)
vectorSize=200

In [0]:
class generator(nn.Module):
    """
    Generator for 3D-GAN-
    The generator consists of five fully convolution layers with numbers of channels
    {512, 256, 128, 64, 1}, kernel sizes {4, 4, 4, 4, 4}, and strides {1, 2, 2, 2, 2}. 
    We add ReLU and batch normalization layers between convolutional layers, and a Sigmoid
    layer at the end. The input is a 200-dimensional vector, and the output is a 64 × 64 × 64 
    matrix with values in [0, 1].
    """
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.ConvTranspose3d(in_channels=200,out_channels=512,kernel_size=2,stride=1,bias=False),
            nn.BatchNorm3d(512),
            nn.ReLU(True),
        )
        self.layer2=nn.Sequential(
            nn.ConvTranspose3d(in_channels=512,out_channels=256,kernel_size=2,stride=2,bias=False),
            nn.BatchNorm3d(256),
            nn.ReLU(True),
        )
        self.layer3=nn.Sequential(
            nn.ConvTranspose3d(in_channels=256,out_channels=128,kernel_size=2,stride=2,bias=False),
            nn.BatchNorm3d(128),
            nn.ReLU(True),
        )
        self.layer4=nn.Sequential(
            nn.ConvTranspose3d(in_channels=128,out_channels=64,kernel_size=2,stride=2,bias=False),
            nn.BatchNorm3d(64),
            nn.ReLU(True),
        )
        self.layer5=nn.Sequential(
            nn.ConvTranspose3d(in_channels=64,out_channels=1,kernel_size=2,stride=2,bias=False),
            nn.Sigmoid(),
        )
    
    def forward(self,x):
        bsize=x.size(0)
        x=x.view(bsize,vectorSize,1,1,1)
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        x=self.layer5(x)
        return x
G_=generator().to(device)
class discriminator(nn.Module):
    """
    Discriminator for 3D-GAN-
    As a mirrored version of the generator, the discriminator takes as input a 
    64 × 64 × 64 matrix, and outputs a real number in [0, 1]. The discriminator 
    consists of 5 volumetric convolution layers, with numbers of channels {64,128,256,512,1}, 
    kernel sizes {4,4,4,4,4}, and strides {2, 2, 2, 2, 1}. There are leaky ReLU layers of 
    parameter 0.2 and batch normalization layers in between, and a Sigmoid layer at the end.
    """
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.Conv3d(in_channels=1,out_channels=64,kernel_size=3,stride=2,bias=False),
            nn.BatchNorm3d(64),
            nn.LeakyReLU(0.2,inplace=True)
        )
        self.layer2=nn.Sequential(
            nn.Conv3d(in_channels=64,out_channels=128,kernel_size=3,stride=2,bias=False),
            nn.BatchNorm3d(128),
            nn.LeakyReLU(0.2,inplace=True)
        )
        self.layer3=nn.Sequential(
            nn.Conv3d(in_channels=128,out_channels=256,kernel_size=3,stride=2,bias=False),
            nn.BatchNorm3d(256),
            nn.LeakyReLU(0.2,inplace=True)
        )
        self.layer4=nn.Sequential(
            nn.Conv3d(in_channels=256,out_channels=512,kernel_size=3,stride=2,bias=False),
            nn.BatchNorm3d(512),
            nn.LeakyReLU(0.2,inplace=True)
        )
        self.layer5=nn.Sequential(
            nn.Conv3d(in_channels=512,out_channels=1,kernel_size=1,stride=1,bias=False),
            nn.Sigmoid()
        )
 
    def forward(self,x):
        bsize=x.size(0)
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        x=self.layer5(x)
        x=x.view(bsize,1)
        return x
D_=discriminator().to(device)

In [0]:
# Number of epochs you want it to train for.
num_epochs=100
# If you want to continue training from perv. saved model, uncomment the lines below.
# G_.load_state_dict(torch.load('./gen'))
# D_.load_state_dict(torch.load('./dis'))

# Defining the optimizers, Adam, with lr as specified in the paper.
optimizerD=optim.Adam(D_.parameters(),lr=1e-6,betas=(0.5,0.999))
optimizerG=optim.Adam(G_.parameters(),lr=0.0025,betas=(0.5,0.999))
# Loss criterion as Binary Cross-Entropy Loss.
criterion=nn.BCELoss()
# Defining real and fake labels.
real_label=1
fake_label=0

In [0]:
# Arrays to store d_losses and g_losses.
G_losses=[]
D_losses=[]
iters = 0
k=10
print("Starting training loop...")
# For every epoch.
for epoch in range(num_epochs):
    # For each batch in epoch.
    RunningLossG=0
    RunningLossD=0
    for i,data in enumerate(data_loader,1):
        ############################
        # Updating Discriminator : maximize log(D(x)) + log(1-D(G(z)))
        ############################
        for j in range(k):
            D_.zero_grad()
            ## Training with all real batch
            real_data = data.to(device).float()
            btSize=real_data.size(0)
            label = torch.full((btSize,),real_label,device=device)
            # Forward pass for real_batch.
            output = D_(real_data).view(-1)
            # Loss calculation for real_batch.
            errD_real = criterion(output,label)
            # Calculating gradients
            errD_real.backward()

            ## Training with all fake batch
            # Generating batch of random noise vectors
            noise = torch.randn((btSize, vectorSize), device=device)
            # print(noise.shape)
            # Generating fake models with G
            fake = G_(noise)
            label.fill_(fake_label)
            output = D_(fake.detach()).view(-1)
            errD_fake=criterion(output,label)
            errD_fake.backward()
            D_G_z1=output.mean().item()
            errD=errD_real+errD_fake
            # Update D network.
            optimizerD.step()
        output = D_(real_data).view(-1)
        D_x = output.mean().item()
        ############################
        # Updating Generator : minimize -log(D(G(z)))
        ############################
        # label.fill_(real_label) # To get the log(x) part rather than log(1-x).
        label.fill_(fake_label)
        G_.zero_grad()
        output=D_(fake).view(-1)
        errG=-criterion(output,label)
        errG.backward()
        D_G_z2=output.mean().item()
        # Update G network
        optimizerG.step()
        RunningLossD+=errD.item()
        RunningLossG+=errG.item()
        if i%7==0:
            print('[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x):%.4f D(G(z)): %.4f/%.4f'%(epoch+1,num_epochs,(errD/k).item(),errG.item(),D_x,D_G_z1,D_G_z2))
    G_losses.append(RunningLossG)
    D_losses.append(RunningLossD/k)

Starting training loop...


/pytorch/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.


[1/100] Loss_D: 0.2915 Loss_G: -2.2921 D(x):0.5482 D(G(z)): 0.5677/0.5677
[1/100] Loss_D: 0.3529 Loss_G: -2.9187 D(x):0.5510 D(G(z)): 0.5533/0.5533
[2/100] Loss_D: 0.3731 Loss_G: -3.1361 D(x):0.5553 D(G(z)): 0.5051/0.5051
[2/100] Loss_D: 0.3297 Loss_G: -2.6882 D(x):0.5595 D(G(z)): 0.4636/0.4639
[3/100] Loss_D: 0.2411 Loss_G: -1.7932 D(x):0.5595 D(G(z)): 0.5213/0.5216
[3/100] Loss_D: 0.2731 Loss_G: -2.1226 D(x):0.5637 D(G(z)): 0.5002/0.5000
[4/100] Loss_D: 0.2971 Loss_G: -2.3658 D(x):0.5625 D(G(z)): 0.5085/0.5085
[4/100] Loss_D: 0.2490 Loss_G: -1.8802 D(x):0.5669 D(G(z)): 0.5117/0.5118
[5/100] Loss_D: 0.2422 Loss_G: -1.8144 D(x):0.5647 D(G(z)): 0.5180/0.5184
[5/100] Loss_D: 0.2363 Loss_G: -1.7553 D(x):0.5695 D(G(z)): 0.5552/0.5549
[6/100] Loss_D: 0.2065 Loss_G: -1.4611 D(x):0.5665 D(G(z)): 0.5274/0.5273
[6/100] Loss_D: 0.2036 Loss_G: -1.4488 D(x):0.5718 D(G(z)): 0.5514/0.5514
[7/100] Loss_D: 0.2045 Loss_G: -1.4441 D(x):0.5683 D(G(z)): 0.5116/0.5124
[7/100] Loss_D: 0.2001 Loss_G: -1.4143

In [0]:
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()
plt.savefig("losses.png")

In [0]:
evalArray=G_(torch.randn((1,200)).to(device))
T=0.8
evalArray[evalArray>T]=True
evalArray[evalArray<T]=False
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.voxels(evalArray.squeeze().detach().cpu().numpy(),facecolors='red')
fig.savefig('thresholded_T={0}.png'.format(T))
plt.show()